## Extract sector table data from etfdb.com

In [18]:
import pandas as pd

In [19]:
url = 'https://etfdb.com/etfs/sector/'
df = pd.read_html(url)[0].drop([11], axis=0)

In [20]:
sectors = list(df['Sector'])
sectors = [x.lower() for x in sectors]
sectors = [x.replace(' ', '-') for x in sectors]
sectors = [x.replace('discretionary', 'discretionaries') for x in sectors]
sectors

['energy',
 'consumer-discretionaries',
 'financials',
 'healthcare',
 'industrials',
 'technology',
 'materials',
 'consumer-staples',
 'telecom',
 'utilities',
 'real-estate']

## Retrieve Top ETFs from Each Sector

In [21]:
top_etfs = {}
for s in sectors:
    sector_url = url + s + '/'
    sector_data = pd.read_html(sector_url)[0]
    etf_symbol = sector_data.loc[0, 'Symbol']
    etf_name = sector_data.loc[0, 'ETF Name']
    etf_industry = sector_data.loc[0, 'Industry']
    etf_aum = sector_data.loc[0, 'Total Assets ($MM)']
    top_etfs[s] = {}
    top_etfs[s]['symbol'] = etf_symbol
    top_etfs[s]['name'] = etf_name
    top_etfs[s]['industry'] = etf_industry
    top_etfs[s]['aum'] = etf_aum

top_etfs

{'energy': {'symbol': 'XLE',
  'name': 'Energy Select Sector SPDR Fund',
  'industry': 'Oil & Gas Exploration & Production',
  'aum': '$22,101.90'},
 'consumer-discretionaries': {'symbol': 'XLY',
  'name': 'Consumer Discretionary Select Sector SPDR Fund',
  'industry': 'Broad Consumer Discretionary',
  'aum': '$19,220.10'},
 'financials': {'symbol': 'XLF',
  'name': 'Financial Select Sector SPDR Fund',
  'industry': 'Broad Financials',
  'aum': '$36,592.80'},
 'healthcare': {'symbol': 'XLV',
  'name': 'Health Care Select Sector SPDR Fund',
  'industry': 'Broad Healthcare',
  'aum': '$24,833.30'},
 'industrials': {'symbol': 'XLI',
  'name': 'Industrial Select Sector SPDR Fund',
  'industry': 'Broad Industrials',
  'aum': '$19,135.00'},
 'technology': {'symbol': 'QQQ',
  'name': 'Invesco QQQ',
  'industry': 'Broad Technology',
  'aum': '$155,470.00'},
 'materials': {'symbol': 'GDX',
  'name': 'VanEck Vectors Gold Miners ETF',
  'industry': 'Gold Miners',
  'aum': '$14,080.20'},
 'consume

## Calculate Sector Momentum

In [41]:
import yfinance as yf
from pandas_datareader.data import DataReader
import datetime as dt

Date
2001-01-31     70.705368
2001-02-28     63.138550
2001-03-31     60.787449
2001-04-30     72.770470
2001-05-31     70.745026
                 ...    
2019-08-31    125.594933
2019-09-30    134.759933
2019-10-31    123.926888
2019-11-30    126.065689
2019-12-31    125.681244
Freq: M, Name: Adj Close, Length: 228, dtype: float64

In [55]:
price_data = {
    'Symbol': [],
    'Sector': [],
    'Industry': [],
    '12 Months': [],
    '6 Months': [],
    '3 Months': [],
    '1 Month': [],
    'Today': []
}

In [56]:
startDate = dt.datetime.today() - dt.timedelta(days=365)
endDate = dt.datetime.today()

for etf in top_etfs.keys():
    symbol = top_etfs[etf]['symbol']
    sector = etf
    industry = top_etfs[etf]['industry']
    daily_price = DataReader(symbol, 'yahoo', startDate, endDate)['Adj Close']
    # print(daily_price)
    today_close = daily_price.iloc[-1]
    # print(today_close)
    # convert daily price to monthly price
    monthly_price = daily_price.resample('M').agg(lambda x: x[-1])
    # drop the last row of monthly_price
    monthly_price.drop(monthly_price.tail(1).index, inplace=True)
    # print(monthly_price)
    prev_one_month = monthly_price.iloc[-1]
    # print(prev_one_month)
    prev_three_month = monthly_price.iloc[-3]
    # print(prev_three_month)
    prev_six_month = monthly_price.iloc[-6]
    prev_year = monthly_price.iloc[-12]

    price_data['Symbol'].append(symbol)
    price_data['Sector'].append(sector)
    price_data['Industry'].append(industry)
    price_data['12 Months'].append(prev_year)
    price_data['6 Months'].append(prev_six_month)
    price_data['3 Months'].append(prev_three_month)
    price_data['1 Month'].append(prev_one_month)
    price_data['Today'].append(today_close)

In [68]:
momentum_df = pd.DataFrame(price_data)
momentum_df

,Symbol,Sector,Industry,12 Months,6 Months,3 Months,1 Month,Today
0,XLE,energy,Oil & Gas Exploration & Production,27.562679,29.244436,37.502102,47.644489,49.029999
1,XLY,consumer-discretionaries,Broad Consumer Discretionary,97.308754,146.475372,160.524994,160.894394,163.259995
2,XLF,financials,Broad Financials,20.374788,23.835936,29.349915,32.167427,33.279999
3,XLV,healthcare,Broad Healthcare,87.193604,104.683556,113.047646,112.220512,114.370003
4,XLI,industrials,Broad Industrials,58.101799,76.427643,88.278305,90.331978,95.129997
5,QQQ,technology,Broad Technology,189.486252,276.987488,313.344238,313.743744,311.869995
6,GDX,materials,Gold Miners,22.920330,38.956600,36.020000,31.129999,32.299999
7,XLP,consumer-staples,Broad Consumer Staples,52.996235,63.225918,67.087242,62.959564,66.739998
8,VOX,telecom,Broad Telecom,75.673325,101.680870,120.199997,128.460007,129.710007
9,XLU,utilities,Broad Utilities,53.625660,58.445972,62.235294,57.927460,62.590000


## Momentum Calculation

In [69]:
momentum_df['M1'] = (momentum_df['Today'] -
                     momentum_df['1 Month']) / momentum_df['1 Month']
momentum_df['M3'] = (momentum_df['Today'] -
                     momentum_df['3 Months']) / momentum_df['3 Months']
momentum_df['M6'] = (momentum_df['Today'] -
                     momentum_df['6 Months']) / momentum_df['6 Months']
momentum_df['M12'] = (momentum_df['Today'] -
                      momentum_df['12 Months']) / momentum_df['12 Months']
momentum_df['EW_Score'] = 0.25 * \
    (momentum_df['M1'] + momentum_df['M3'] +
     momentum_df['M6'] + momentum_df['M12'])

## Final DataFrame with Equal Weight Momentum Score

In [75]:
momentum_df.set_index('Symbol', inplace=True)
momentum_df.sort_values(by='EW_Score', ascending=True)

,Sector,Industry,12 Months,6 Months,3 Months,1 Month,Today,M1,M3,M6,M12,EW_Score
Symbol,,,,,,,,,,,,
GDX,materials,Gold Miners,22.920330,38.956600,36.020000,31.129999,32.299999,0.037584,-0.103276,-0.170872,0.409229,0.043166
XLU,utilities,Broad Utilities,53.625660,58.445972,62.235294,57.927460,62.590000,0.080489,0.005699,0.070904,0.167165,0.081064
XLP,consumer-staples,Broad Consumer Staples,52.996235,63.225918,67.087242,62.959564,66.739998,0.060045,-0.005176,0.055580,0.259335,0.092446
XLV,healthcare,Broad Healthcare,87.193604,104.683556,113.047646,112.220512,114.370003,0.019154,0.011697,0.092531,0.311679,0.108765
VNQ,real-estate,Broad Real Estate,67.546326,77.709572,84.930000,87.870003,90.489998,0.029817,0.065466,0.164464,0.339673,0.149855
QQQ,technology,Broad Technology,189.486252,276.987488,313.344238,313.743744,311.869995,-0.005972,-0.004705,0.125935,0.645871,0.190282
XLY,consumer-discretionaries,Broad Consumer Discretionary,97.308754,146.475372,160.524994,160.894394,163.259995,0.014703,0.017038,0.114590,0.677752,0.206021
XLI,industrials,Broad Industrials,58.101799,76.427643,88.278305,90.331978,95.129997,0.053115,0.077615,0.244707,0.637299,0.253184
VOX,telecom,Broad Telecom,75.673325,101.680870,120.199997,128.460007,129.710007,0.009731,0.079118,0.275658,0.714078,0.269646


Based on Equal Weight Momentum Score, energy sector has the strongest momentum and materials sector has the weakest momentum. There are two perspectives we can build. We can invest in the companies in the sector with the strongest momentum expecting the strong momentum continues, or we can look for companies in the sector with lower momentum score expecting the prices to rebound.